In [1]:
from pathlib import Path, PurePath
import pandas as pd
import matplotlib.pyplot as plt
import os
import obspy
from preprocessing_tools import *

1 día de datos, primeros 4 dígitos estación, y luego el canal después del punto. Hay un evento significativo de magnitud mayor a 5 que puede usarse como referencia para probar la buena implementación del método de John Ebel y Nawa Dahal.

El archivo catalog.dat sirve para especificar los eventos sísmicos significativos que fueron detectados en el conjunto grande de datos y sus columnas son: fecha, hora, latitud, longitud, profundidad, magnitud y lasa ultimas tres columnas son datos independientes relacionados con la similitud o correlación entre dichos eventos. A partir de este archivo, es posible localizar y filtrar los datos completos para incluir únicamente la sección en que se percibieron estos eventos para tratar menos datos. Posibilidad de paralelizar en caso de que se requiera.

Una vez hecho esto, ya toca implementar el método de John Ebel y Nawa Dahal en cuestión

In [2]:
directorio_completo = r'Datos/'
directorio_recortado = r'Datos_Trim/'
if not os.path.exists(directorio_recortado):
    os.makedirs(directorio_recortado)

In [3]:
catalog = 'catalog.dat'

In [4]:
df_catalog = read_catalog(catalog)

In [5]:
df_catalog.head()

,date_time,lat,lon,depth,mag,corr
0,2018-08-12T01:17:47.060000Z,17.1800,-100.9500,3.10,1.20,"[0.7309, 37.0600, 20180812011747.06]"
1,2018-08-12T01:37:43.000000Z,17.1800,-101.0200,26.90,3.90,"[1.0000, 52.1500, 20180812013743.00]"
2,2018-08-12T02:06:47.640000Z,17.2350,-101.4350,26.00,2.41,"[0.5361, 22.2400, 20180605040434.00]"
3,2018-08-12T02:21:18.100000Z,17.0300,-100.9000,69.70,2.25,"[0.5164, 21.0200, 20180810013340.00]"
4,2018-08-12T05:13:06.000000Z,16.4900,-100.5800,6.40,4.30,"[0.9795, 46.3900, 20180812051306.00]"


In [6]:
df_sort = df_catalog.sort_values('mag', ascending=False).copy()

In [7]:
df_sort.head()

,date_time,lat,lon,depth,mag,corr
8,2018-08-12T14:42:09.000000Z,17.1100,-100.8400,44.30,5.20,"[0.9316, 51.6100, 20180812144209.00]"
4,2018-08-12T05:13:06.000000Z,16.4900,-100.5800,6.40,4.30,"[0.9795, 46.3900, 20180812051306.00]"
1,2018-08-12T01:37:43.000000Z,17.1800,-101.0200,26.90,3.90,"[1.0000, 52.1500, 20180812013743.00]"
6,2018-08-12T11:45:10.000000Z,16.9500,-100.3800,31.90,3.60,"[0.9133, 45.2600, 20180812114510.00]"
2,2018-08-12T02:06:47.640000Z,17.2350,-101.4350,26.00,2.41,"[0.5361, 22.2400, 20180605040434.00]"


In [8]:
t0 = df_sort['date_time'][0] # tiempo de sismo mas fuerte registrado
tf = t0 + 60
print(t0)

2018-08-12T01:17:47.060000Z


In [9]:
stations_meta = get_meta_data(directorio_completo)

In [10]:
stations_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 35 columns):
b           21 non-null float64
cmpaz       21 non-null float64
cmpinc      21 non-null float64
delta       21 non-null float64
depmax      21 non-null float64
depmen      3 non-null float64
depmin      21 non-null float64
e           21 non-null float64
iftype      21 non-null int64
kcmpnm      21 non-null object
kevnm       21 non-null object
kstnm       21 non-null object
kuser0      21 non-null object
lcalda      21 non-null int64
leven       21 non-null int64
lovrok      21 non-null int64
lpspol      21 non-null int64
nevid       21 non-null int64
norid       21 non-null int64
npts        21 non-null int64
nvhdr       21 non-null int64
nzhour      21 non-null int64
nzjday      21 non-null int64
nzmin       21 non-null int64
nzmsec      21 non-null int64
nzsec       21 non-null int64
nzyear      21 non-null int64
resp0       21 non-null float64
resp1       21 non-null float64
resp

In [11]:
stations_meta['kstnm']

0     1S01
1     1S01
2     1S01
3     1S03
4     1S03
5     1S03
6     1S04
7     1S04
8     1S04
9     1S05
10    1S05
11    1S05
12    1S07
13    1S07
14    1S07
15    1S08
16    1S08
17    1S08
18    1S09
19    1S09
20    1S09
Name: kstnm, dtype: object

In [12]:
stations_meta.head(30) # stel dejar tal cual o a 0
# No unir en grupos las filas ya que las primeras columnas son de orientacion de los sensores de cada canal, lo cual es importante
# Crear repositorio github

,b,cmpaz,cmpinc,delta,depmax,depmen,depmin,e,iftype,kcmpnm,...,nzsec,nzyear,resp0,resp1,resp2,resp3,stel,stla,stlo,unused23
0,0.0,90.0,90.0,0.005,1.712276e-03,0.000001,-1.712279e-03,86400.0,1,E,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0
1,0.0,0.0,90.0,0.005,1.712276e-03,0.000001,-1.712279e-03,86400.0,1,N,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0
2,0.0,0.0,0.0,0.005,1.344283e-03,0.000002,-9.943596e-04,86400.0,1,U,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0
3,0.0,90.0,90.0,0.005,3.318486e+38,NaN,-3.358415e+38,86400.0,1,E,...,0,2018,3.265913e-09,20.0,1.0,0.7,-984.0,16.797607,-100.459198,0
4,0.0,0.0,90.0,0.005,3.343357e+38,NaN,-3.379704e+38,86400.0,1,N,...,0,2018,3.265913e-09,20.0,1.0,0.7,-984.0,16.797607,-100.459198,0
5,0.0,0.0,0.0,0.005,3.391023e+38,NaN,-3.383286e+38,86400.0,1,U,...,0,2018,3.265913e-09,20.0,1.0,0.7,-984.0,16.797607,-100.459198,0
6,0.0,90.0,90.0,0.005,3.346421e+38,NaN,-3.365736e+38,86400.0,1,E,...,0,2018,3.265913e-09,20.0,1.0,0.7,-1388.0,16.923592,-100.757507,0
7,0.0,0.0,90.0,0.005,3.339749e+38,NaN,-3.358415e+38,86400.0,1,N,...,0,2018,3.265913e-09,20.0,1.0,0.7,-1388.0,16.923592,-100.757507,0
8,0.0,0.0,0.0,0.005,3.343357e+38,NaN,-3.400317e+38,86400.0,1,U,...,0,2018,3.265913e-09,20.0,1.0,0.7,-1388.0,16.923592,-100.757507,0
9,0.0,90.0,90.0,0.005,3.343331e+38,NaN,-3.386350e+38,86400.0,1,E,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2347.0,16.801371,-100.998512,0


In [13]:
stations_meta.loc[stations_meta['kstnm'] == '1S01']

,b,cmpaz,cmpinc,delta,depmax,depmen,depmin,e,iftype,kcmpnm,...,nzsec,nzyear,resp0,resp1,resp2,resp3,stel,stla,stlo,unused23
0,0.0,90.0,90.0,0.005,0.001712,0.000001,-0.001712,86400.0,1,E,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0
1,0.0,0.0,90.0,0.005,0.001712,0.000001,-0.001712,86400.0,1,N,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0
2,0.0,0.0,0.0,0.005,0.001344,0.000002,-0.000994,86400.0,1,U,...,0,2018,3.265913e-09,20.0,1.0,0.7,-2386.0,16.683641,-100.741394,0


In [14]:
Trim_Org(directorio_recortado, directorio_completo, 60, df_catalog, stations_meta)

No hay que hacer group by ya que cada canal de cada estacion tiene ciertas alineaciones o angulos distintos

Descuidar de altitudes negativas, estblecer en 0 si se da error

In [16]:
directorio_evento = directorio_recortado + datetime_to_dotformat(df_catalog.values[0][0])+'/'
tr_df, tr_st = get_tr_meta(directorio_evento)

In [17]:
tr_df.head()

,starttime,endtime,sampling_rate,id
0,2018-08-12T01:17:47.060059Z,2018-08-12T01:18:47.060059Z,200.0,0
1,2018-08-12T01:17:47.060059Z,2018-08-12T01:18:47.060059Z,200.0,1
2,2018-08-12T01:17:47.060059Z,2018-08-12T01:18:47.060059Z,200.0,2
3,2018-08-12T01:17:47.060059Z,2018-08-12T01:18:47.060059Z,200.0,3
4,2018-08-12T01:17:47.060059Z,2018-08-12T01:18:47.060059Z,200.0,4


In [19]:
tr_st.head(20)

,station,channel,tr_id
0,1S01,E,0
1,1S01,N,1
2,1S01,U,2
3,1S03,E,3
4,1S03,N,4
5,1S03,U,5
6,1S04,E,6
7,1S04,N,7
8,1S04,U,8
9,1S05,E,9


In [20]:
#station1 = stations_meta.groupby('kstnm').apply(list).reset_index()

In [21]:
#station1.head()

In [22]:
#stations_meta.loc[stations_meta['kstnm'] == '1S01'].iloc[0]

In [21]:
#stations_meta.loc[stations_meta['kstnm'] == '1S01'].iloc[1]

In [23]:
#A = obspy.read('Datos_Trim/*')

#A.plot()
#plt.show()

En desarrollo...

In [2]:
for index,row in df_catalog.iterrows():
    event_dir = directorio_recortado
    outdir = "%s/sac"%event_dir # Output directory
    
    st = obspy.read(directorio_recortado+'/*', format='SAC')

    # Detrend and remove instrument response
    st.detrend(type="linear") # equivalent to rtr in SAC
    st.remove_response(inventory=inv, pre_filt=pre_filt, output="DISP", zero_mean=True) # correct to displacement
    st.detrend(type="linear")
    st.detrend(type="demean") # remove mean

    # Define SAC headers and calculate back-azimtuh for rotation
    origin_time = UTCDateTime(row.date_time)
    depth = row.depth*1000
    distances = []
    azimuths = []
    b_azimuths = []
    o = []
    b = []
    for i, tr in tr_df.iterrows():
        #meta = inv.get_channel_metadata(tr.id)
        dist, az, baz = gps2dist_azimuth(row.lat,row.lon,sta.stla,sta.stlo)
        omarker = origin_time - tr.starttime
        
        # baz (back azimuth) es aquel angulo de la estacion con las coordenadas del sismo o evento, asi que lo tendremos que guardar en el trace 

        # Obspy trace headers
        #tr.stats.coordinates = {'latitude':meta['latitude'], 'longitude':meta['longitude']}
        distances.append(dist/1000) # A kilometros
        azimuths.append(az)
        b_azimuths.append(baz)
        o.append(0)
        b.append(-1*omarker)
        
        # Modificando valores en los traces (puede que haya que reemplazar los archivos o volver a guardarlos para que sean efectivos los cambios)
        st[i].stats.sac.az = az
        st[i].stats.sac.baz = baz
        st[i].stats.sac.dist = dist/1000
        st[i].stats.sac.o = 0
        st[i].stats.sac.b = -1*omarker
        
    tr_df['distance'] = distances
    tr_df['azimuth'] = azimuths
    tr_df['b_azimuth'] = b_azimuths
    tr_df['o'] = o
    tr_df['b'] = b
    
    # No es necesario rotar a ZNE en el caso de los datos de la red subarina ya que los tres componentes o canales ya vienen orientados en norte, este y la vertical

    # Todas las estaciones submarinas ya vienen con tres componentes, asi que tampoco es necesario verificarlo
    # Se rota a RT para trabajar mas facilmente la distancia y el angulo con cada sismo
    st.rotate(method="NE->RT")

NameError: name 'df_catalog' is not defined

Entre las cosas que podrian faltarle a esta libreta aun, es el filtrado de los sismogramas por frecuencia.